In [ ]:
import json
import pandas as pd
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#load credentials
json_path = "/opt/airflow/data/breast_cancer_data/credentials.json"

with open(json_path, "r", encoding="utf-8") as file:
    creds = json.load(file)

print("Credentials Loaded Successfully!", creds)

MSSQL_HOST = creds["MSSQL_HOST"]
MSSQL_DATABASE = creds["MSSQL_DATABASE"]
MSSQL_USER = creds["MSSQL_USER"]
MSSQL_PASSWORD = creds["MSSQL_PASSWORD"]
MSSQL_PORT = creds["MSSQL_PORT"]

In [ ]:
#connect to database
conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER=host.docker.internal,1433;"  # Using host.docker.internal for Docker access
    f"DATABASE={MSSQL_DATABASE};"
    f"UID={MSSQL_USER};"
    f"PWD={MSSQL_PASSWORD};"
    f"TrustServerCertificate=yes;"
    f"Connection Timeout=30;"
)

cursor = conn.cursor()

In [17]:
#load cleaned data
CLEANED_CSV_PATH = "/opt/airflow/data/breast_cancer_data/breast_cancer_data_cleaned.csv"
df = pd.read_csv(CLEANED_CSV_PATH)

In [ ]:
# Prepare Features and Target
X = df.drop(["id", "diagnosis"], axis=1)  # Features
y = df["diagnosis"]  # Target Variable

# Convert Diagnosis to Binary (0 for B, 1 for M)
y = y.apply(lambda x: 1 if x == "M" else 0)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Predict Class Labels
y_pred = model.predict(X_test)  # Binary Predictions

# Predict Probabilities (NEW STEP!)
y_proba = model.predict_proba(X_test)[:, 1]  # Probability of being Malignant
y_pred_labels = ["M" if pred == 1 else "B" for pred in y_pred]

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model trained with {accuracy * 100:.2f}% accuracy")

# Store Predictions with Probabilities
test_data = X_test.copy()
test_data["id"] = df.loc[X_test.index, "id"]  # Restore ID column
test_data["diagnosis"] = y_test  # Actual labels
test_data["prediction"] = y_pred_labels  # Predicted class (B/M)
test_data["probability"] = y_proba  # Probability of being Malignant

print(test_data.head())  # Check if probabilities are correctly stored


# **Rearrange Columns to Match SQL Table Order**
column_order = [
    "id", "diagnosis", "radius_mean", "texture_mean", "perimeter_mean", "area_mean",
    "smoothness_mean", "compactness_mean", "concavity_mean", "concave_points_mean",
    "symmetry_mean", "fractal_dimension_mean", "radius_se", "texture_se",
    "perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se",
    "concave_points_se", "symmetry_se", "fractal_dimension_se", "radius_worst",
    "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst",
    "fractal_dimension_worst", "prediction", "probability"
]

test_data = test_data[column_order]  # Ensure column order matches SQL table

In [ ]:
# Store Results in SQL Server
# Store the Cleaned Dataset in SQL Server
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'breast_cancer_cleaned')
    CREATE TABLE breast_cancer_cleaned (
        id INT PRIMARY KEY,
        diagnosis VARCHAR(2),
        radius_mean FLOAT, texture_mean FLOAT, perimeter_mean FLOAT, area_mean FLOAT,
        smoothness_mean FLOAT, compactness_mean FLOAT, concavity_mean FLOAT, concave_points_mean FLOAT,
        symmetry_mean FLOAT, fractal_dimension_mean FLOAT, radius_se FLOAT, texture_se FLOAT,
        perimeter_se FLOAT, area_se FLOAT, smoothness_se FLOAT, compactness_se FLOAT, concavity_se FLOAT,
        concave_points_se FLOAT, symmetry_se FLOAT, fractal_dimension_se FLOAT, radius_worst FLOAT,
        texture_worst FLOAT, perimeter_worst FLOAT, area_worst FLOAT, smoothness_worst FLOAT,
        compactness_worst FLOAT, concavity_worst FLOAT, concave_points_worst FLOAT, symmetry_worst FLOAT,
        fractal_dimension_worst FLOAT
    );
""")
conn.commit()

# Insert Cleaned Data into SQL Server
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO breast_cancer_cleaned 
        (id, diagnosis, radius_mean, texture_mean, perimeter_mean, area_mean,
        smoothness_mean, compactness_mean, concavity_mean, concave_points_mean,
        symmetry_mean, fractal_dimension_mean, radius_se, texture_se,
        perimeter_se, area_se, smoothness_se, compactness_se, concavity_se,
        concave_points_se, symmetry_se, fractal_dimension_se, radius_worst,
        texture_worst, perimeter_worst, area_worst, smoothness_worst,
        compactness_worst, concavity_worst, concave_points_worst, symmetry_worst,
        fractal_dimension_worst)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))
conn.commit()

# now store predictions
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'breast_cancer_predictions')
    CREATE TABLE breast_cancer_predictions (
        id INT PRIMARY KEY,
        diagnosis varchar(2),
        radius_mean FLOAT, texture_mean FLOAT, perimeter_mean FLOAT, area_mean FLOAT,
        smoothness_mean FLOAT, compactness_mean FLOAT, concavity_mean FLOAT, concave_points_mean FLOAT,
        symmetry_mean FLOAT, fractal_dimension_mean FLOAT, radius_se FLOAT, texture_se FLOAT,
        perimeter_se FLOAT, area_se FLOAT, smoothness_se FLOAT, compactness_se FLOAT, concavity_se FLOAT,
        concave_points_se FLOAT, symmetry_se FLOAT, fractal_dimension_se FLOAT, radius_worst FLOAT,
        texture_worst FLOAT, perimeter_worst FLOAT, area_worst FLOAT, smoothness_worst FLOAT,
        compactness_worst FLOAT, concavity_worst FLOAT, concave_points_worst FLOAT, symmetry_worst FLOAT,
        fractal_dimension_worst FLOAT, prediction varchar(2), probability FLOAT
    );
""")
conn.commit()

# Insert Data into Table
for _, row in test_data.iterrows():
    cursor.execute("""
        INSERT INTO breast_cancer_predictions 
        (id, diagnosis, radius_mean, texture_mean, perimeter_mean, area_mean,
        smoothness_mean, compactness_mean, concavity_mean, concave_points_mean,
        symmetry_mean, fractal_dimension_mean, radius_se, texture_se,
        perimeter_se, area_se, smoothness_se, compactness_se, concavity_se,
        concave_points_se, symmetry_se, fractal_dimension_se, radius_worst,
        texture_worst, perimeter_worst, area_worst, smoothness_worst,
        compactness_worst, concavity_worst, concave_points_worst, symmetry_worst,
        fractal_dimension_worst, prediction, probability)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))
conn.commit()

In [ ]:
# Close Connection
cursor.close()
conn.close()

print("Predictions saved to MS SQL Server successfully!")